# Librerias

In [15]:
import pandas as pd
import numpy as np
import time
import sys
import random

# Lectura de Datos

In [16]:
archivo=pd.read_csv('Parallel_Machine_instances/Instance 3.txt', header=None, delim_whitespace=True)
tamaño_archivo=len(archivo)
numero_columnas=len(archivo.columns)

In [17]:
numero_tareas=int(archivo.iloc[-2][0])
numero_maquinas=int(archivo.iloc[-2][1])
matriz_tiempos=[]
for i in range(numero_tareas):
    j=1
    while j<(numero_columnas):
        matriz_tiempos.append(archivo.iloc[i][j])
        j+=2
    
matriz_tiempos = np.asarray(matriz_tiempos)
matriz_tiempos= matriz_tiempos.astype(int)
matriz_tiempos=matriz_tiempos.reshape(numero_tareas,numero_maquinas)


# Solución inicial Greedy

A cada tarea se asigna a la maquina con menor tiempo de ejecución

In [18]:
asignacion_maquinas={}
tiempo_maquina={}
for i in range(numero_maquinas):
    asignacion_maquinas[i]=[]

for i in range(numero_maquinas):
    tiempo_maquina[i]=[]
 
for i in range(numero_tareas):
    maquina_asignar=np.argmin(matriz_tiempos[i])
    tiempo_maquina[maquina_asignar].append(np.min(matriz_tiempos[i]))
    asignacion_maquinas[maquina_asignar].append(i)
print("Tareas asignadas a maquinas= ",asignacion_maquinas)
print("tiempo de uso cada maquina= ",tiempo_maquina)

Tareas asignadas a maquinas=  {0: [16, 23, 51, 67, 68, 85], 1: [0, 6, 17, 20, 26, 28, 33, 46, 61, 64, 65, 73], 2: [10, 27, 29, 42, 59, 66, 72, 83, 86, 89, 94, 96], 3: [5, 8, 19, 49, 56, 75, 77, 98], 4: [1, 21, 22, 32, 37, 52, 54, 79, 93, 99], 5: [2, 15, 25, 38, 48, 58, 62, 92], 6: [18, 30, 31, 36, 57, 60, 63, 70, 71, 74, 78, 82], 7: [3, 13, 14, 24, 35, 43, 47, 55, 69, 76, 84, 88], 8: [7, 11, 39, 41, 44, 45, 53, 81, 87, 90, 91, 95, 97], 9: [4, 9, 12, 34, 40, 50, 80]}
tiempo de uso cada maquina=  {0: [14, 7, 9, 11, 25, 17], 1: [10, 5, 25, 5, 7, 14, 13, 4, 21, 6, 9, 31], 2: [8, 5, 2, 12, 30, 7, 1, 3, 6, 5, 11, 7], 3: [8, 10, 37, 8, 11, 3, 1, 3], 4: [8, 19, 9, 14, 5, 11, 12, 8, 10, 9], 5: [7, 1, 2, 14, 14, 10, 30, 24], 6: [12, 16, 26, 4, 17, 9, 26, 8, 3, 18, 14, 9], 7: [7, 13, 2, 35, 2, 16, 2, 23, 8, 14, 3, 1], 8: [15, 10, 1, 5, 16, 3, 11, 10, 15, 6, 2, 11, 13], 9: [9, 6, 7, 11, 6, 13, 5]}


# Funcion Fitness

In [19]:
def evaluacion(numero_maquinas,tiempo_maquina):
    tiempos_final_maquina=[]
    for i in range(numero_maquinas):
        tiempos_final_maquina.append(int(sum(tiempo_maquina[i])))    
    makespan=max(tiempos_final_maquina)
    return(makespan,tiempos_final_maquina)


In [20]:
makespan,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
print(makespan)
print(tiempos_final_maquina)

162
[83, 150, 97, 81, 105, 102, 162, 126, 118, 57]


# Funcion busca maquinas por tarea

In [21]:
def maquina_por_tarea(tarea):
    for i in range(numero_maquinas):
        for j in asignacion_maquinas[i]:
            if tarea==j:
                return(i)

# Funcion mejor cambio de maquina

In [22]:
def mejor_cambio_maquina(tarea,tiempos_final_maquina):
    tiempos=[]
    for i in range(numero_maquinas):
        tiempos.append(tiempos_final_maquina[i]+matriz_tiempos[tarea][i])
    tiempos=np.asarray(tiempos)
    tiempo=np.min(tiempos)
    return(tiempo,np.argmin(tiempos))    

# Funcion para actualizar solucion

In [23]:
def actualizar_solucion(asignacion_maquinas,tiempo_maquina,tarea,maquina_vieja,maquina_nueva,matriz_tiempos):
    asignacion_maquinas[maquina_vieja].remove(tarea)
    asignacion_maquinas[maquina_nueva].append(tarea)
    tiempo_maquina[maquina_vieja].remove(matriz_tiempos[tarea][maquina_vieja])
    tiempo_maquina[maquina_nueva].append(matriz_tiempos[tarea][maquina_nueva])
    return(asignacion_maquinas,tiempo_maquina)

# Iterated Local Search

## Insertion local search

In [24]:
def insertion_local_search(pi,asignacion_maquinas,tiempos_final_maquina,tiempo_maquina):
    improved=True
    while improved:
        improved=False
        for j in range(numero_tareas):                
            i=maquina_por_tarea(j)
            l,maquina_nueva=mejor_cambio_maquina(j,tiempos_final_maquina)
            if l<tiempos_final_maquina[i]:
                improved=True
                asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,j,i \
                                                                       ,maquina_nueva,matriz_tiempos)
                pi,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
    return(pi,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina)
                

## Interchange Local Search

In [25]:
def interchange_local_search(pi_prima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina):
    pi=pi_prima
    improved=True
    while improved:
        improved=False
        for j_1 in range(numero_tareas):
            for j_2 in range(numero_tareas):
                i_1=maquina_por_tarea(j_1)
                i_2=maquina_por_tarea(j_2)
                if j_1!=j_2 and i_1!=i_2 and matriz_tiempos[j_2][i_1]+matriz_tiempos[j_1][i_2]< \
                matriz_tiempos[j_1][i_1]+matriz_tiempos[j_2][i_2] and (tiempos_final_maquina[i_1]+matriz_tiempos[j_2][i_1] \
                - matriz_tiempos[j_1][i_1])<c_max and (tiempos_final_maquina[i_2]+matriz_tiempos[j_1][i_2] \
                - matriz_tiempos[j_2][i_2])<c_max:
                    asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,j_1,i_1 \
                                                                           ,i_2,matriz_tiempos)
                    asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,j_2,i_2 \
                                                                           ,i_1,matriz_tiempos) 
                    pi,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
                    improved=True
    return(pi,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina)               

## Solution Modification

In [ ]:
def solution_modification(pi,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina,matriz_tiempos):
    d=5
    for i in range(d): 
        maquina_random=random.randint(0,9)
        trabajo_random=random.choice(asignacion_maquinas[maquina_random])
        l,maquina_nueva=mejor_cambio_maquina(trabajo_random,tiempos_final_maquina)
        asignacion_maquinas,tiempo_maquina=actualizar_solucion(asignacion_maquinas,tiempo_maquina,trabajo_random,maquina_random,maquina_nueva,matriz_tiempos)   
        pi,tiempos_final_maquina=evaluacion(numero_maquinas,tiempo_maquina)
    return(pi,asignacion_maquinas,tiempo_maquina,tiempos_final_maquina)

## Main

In [ ]:
pi=makespan
pi_best=pi
c_max=pi
tiempo_ejecucion=1800
start=time.time()
end=0
fin=end-start
while fin <= tiempo_ejecucion: 
    improved=True
    while improved:
        improved=False
        pi_prima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina=insertion_local_search(pi,asignacion_maquinas,tiempos_final_maquina,tiempo_maquina)
        c_max=pi_prima
        pi_dosprima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina=interchange_local_search(pi_prima,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina)
        if pi_dosprima!=pi_prima:
            improved=True
            pi=pi_dosprima
    if pi_dosprima<=pi_best:
        pi_best=pi_dosprima
        pi,asignacion_maquinas,tiempo_maquina,tiempos_final_maquina=solution_modification(pi_best,tiempos_final_maquina,asignacion_maquinas,tiempo_maquina,matriz_tiempos)
    end=time.time()
    fin=end-start
print("makespan= ",pi)
print("trabajos asignados= ",asignacion_maquinas)



# Salida de datos

In [ ]:
for i in range(numero_maquinas):
    print("Maquina"+str(i)+"        "+"Trabajos Asignados=",asignacion_maquinas.get(i))

In [ ]:
print(pi)